In [3]:
import os
import glob
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import timm
from tqdm import tqdm

import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import mean_absolute_error
from datetime import datetime

from inference.models import AgeEstimatorModel

2024-01-28 19:20:36.453096: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-28 19:20:36.475475: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-28 19:20:36.475508: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-28 19:20:36.475551: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-28 19:20:36.481071: I tensorflow/core/platform/cpu_feature_g

In [4]:
SIZE=48
BATCH_SIZE=200
EPOCHS=80
LR=0.0005

DATASET_PATH="../data/UTKFace_48"
LOG_PATH="../../logs/age"
MODEL_PATH="../../models/age_model_torch.pth"
WEIGHTS_PATH="../../models/age_model_weights.pth"
TEST_IMAGE_PATH="../data/face_recognition_images/person1.1.jpg"
TIME_FORMAT="%d-%m-%Y; %H:%M:%S"

In [5]:
transform = transforms.Compose([
    transforms.Resize((SIZE, SIZE)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = AgeEstimatorModel().to(device)

In [11]:
class UTKFaceDataset(Dataset):
    def __init__(self, directory, transform=None):
        self.files = glob.glob(os.path.join(directory, '*.jpg'))
        self.transform = transform

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        img_name = self.files[idx]
        image = Image.open(img_name)
        filename = img_name.split('/')[-1]
        age = int(filename.split('_')[0])

        if self.transform:
            image = self.transform(image)

        return image, age

In [12]:
dataset = UTKFaceDataset(directory='../data/UTKFace_48', transform=transform)
train_size = int(0.85 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [13]:
# Использование встроенной функции для анализа классов
class_counts = torch.zeros(90)

for _, labels in train_loader:
    class_counts += torch.bincount(labels, minlength=90)

print("Количество экземпляров каждого класса:")
for i, count in enumerate(class_counts):
    print(f"Класс {i}: {int(count)} экземпляров")

Количество экземпляров каждого класса:
Класс 0: 0 экземпляров
Класс 1: 0 экземпляров
Класс 2: 415 экземпляров
Класс 3: 236 экземпляров
Класс 4: 237 экземпляров
Класс 5: 167 экземпляров
Класс 6: 115 экземпляров
Класс 7: 124 экземпляров
Класс 8: 230 экземпляров
Класс 9: 144 экземпляров
Класс 10: 133 экземпляров
Класс 11: 52 экземпляров
Класс 12: 108 экземпляров
Класс 13: 70 экземпляров
Класс 14: 134 экземпляров
Класс 15: 149 экземпляров
Класс 16: 211 экземпляров
Класс 17: 136 экземпляров
Класс 18: 221 экземпляров
Класс 19: 84 экземпляров
Класс 20: 239 экземпляров
Класс 21: 291 экземпляров
Класс 22: 342 экземпляров
Класс 23: 359 экземпляров
Класс 24: 742 экземпляров
Класс 25: 633 экземпляров
Класс 26: 1863 экземпляров
Класс 27: 518 экземпляров
Класс 28: 773 экземпляров
Класс 29: 478 экземпляров
Класс 30: 626 экземпляров
Класс 31: 302 экземпляров
Класс 32: 556 экземпляров
Класс 33: 125 экземпляров
Класс 34: 350 экземпляров
Класс 35: 754 экземпляров
Класс 36: 403 экземпляров
Класс 37: 250 э

In [15]:
loss_fun = nn.MSELoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=LR)

In [16]:
writer = SummaryWriter(log_dir=LOG_PATH + "/" + datetime.now().strftime(TIME_FORMAT))

for epoch in tqdm(range(EPOCHS)):  # проход по датасету несколько раз
    model.train()
    running_loss = 0.0
    for i, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_fun(outputs.view(-1), labels.float())
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    writer.add_scalar('Metrics/epoch_loss', running_loss  / len(train_loader), epoch)

    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for i, (images, labels) in enumerate(test_loader):
            images, labels = images.to(device), labels
            outputs = model(images)
            all_preds.extend(outputs.tolist())
            all_labels.extend(labels.tolist())
    err = mean_absolute_error(all_labels, all_preds)
    writer.add_scalar('Metrics/MAE', err, epoch)

print('Finished Training')

100%|██████████| 80/80 [24:49<00:00, 18.62s/it]

Finished Training


In [17]:
torch.save(model, MODEL_PATH)
torch.save(model.state_dict(), WEIGHTS_PATH)

In [7]:
# model = torch.load(MODEL_PATH)
model.load_state_dict(torch.load(WEIGHTS_PATH))
model.eval()

image = Image.open(TEST_IMAGE_PATH)
image = transform(image)
image = image.to(device)
image = image.unsqueeze(0)

with torch.no_grad():
    output = model(image)
    predicted_age = output.item()

print(f'Predicted Age: {predicted_age}')

Predicted Age: 32.08957290649414
